# Make Test Network

In [1]:
import os
import sys
import yaml
import glob
import copy

import pandas as pd
import geopandas as gpd

from network_wrangler import RoadwayNetwork
from network_wrangler import TransitNetwork
from network_wrangler import WranglerLogger

from lasso import ModelRoadwayNetwork
from lasso import StandardTransit
from lasso import Parameters
from lasso import mtc

In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
import logging
logger = logging.getLogger("WranglerLogger")
logger.handlers[0].stream = sys.stdout
logger.setLevel(logging.INFO)

### Remote I/O and Parameters

In [4]:
# root_dir = os.path.join('/Users', 'wsp', 'Documents', 'GitHub', 'travel-model-two-networks')
root_dir = os.path.join('C:\\Users', 'david.ory', 'Documents', 'GitHub', 'travel-model-two-networks')
input_dir = os.path.join(root_dir, 'data', 'processed', 'version_03')
boundary_dir = os.path.join(root_dir, 'data', 'external', 'census')

# lasso_dir = os.path.join('/Users', 'wsp', 'Documents', 'GitHub', 'mtc-Lasso')
lasso_dir = os.path.join('C:\\Users', 'david.ory', 'Documents', 'GitHub', 'mtc_Lasso')
output_dir = os.path.join(lasso_dir, "examples", "mtc")

In [6]:
parameters = Parameters(lasso_base_dir = lasso_dir)

2021-01-26 11:36:05, INFO: Lasso base directory set as: Z:/Data/Users/Sijia/MTC/github/Lasso
2021-01-26 11:36:05, INFO: Lasso base directory set as: Z:/Data/Users/Sijia/MTC/github/Lasso
2021-01-26 11:36:05, INFO: Lasso base directory set as: Z:/Data/Users/Sijia/MTC/github/Lasso
2021-01-26 11:36:05, INFO: Lasso base directory set as: Z:/Data/Users/Sijia/MTC/github/Lasso


### Data Reads

In [7]:
link_file = os.path.join(input_dir, 'v03_link.json')
node_file = os.path.join(input_dir, 'v03_node.geojson')
shape_file = os.path.join(input_dir, 'v03_shape.geojson')

road_net = RoadwayNetwork.read(
    link_filename = link_file, 
    node_filename = node_file, 
    shape_filename = shape_file
)

2021-01-26 11:36:05, INFO: Reading RoadwayNetwork
2021-01-26 11:36:05, INFO: Reading RoadwayNetwork
2021-01-26 11:41:06, INFO: Read 2230053 links from D:/gitHub/travel-model-two-networks\data\processed\version_03\v03_link.json
2021-01-26 11:41:06, INFO: Read 2230053 links from D:/gitHub/travel-model-two-networks\data\processed\version_03\v03_link.json
2021-01-26 11:41:06, INFO: Read 696464 nodes from D:/gitHub/travel-model-two-networks\data\processed\version_03\v03_node.geojson
2021-01-26 11:41:06, INFO: Read 696464 nodes from D:/gitHub/travel-model-two-networks\data\processed\version_03\v03_node.geojson
2021-01-26 11:41:06, INFO: Read 1169824 shapes from D:/gitHub/travel-model-two-networks\data\processed\version_03\v03_shape.geojson
2021-01-26 11:41:06, INFO: Read 1169824 shapes from D:/gitHub/travel-model-two-networks\data\processed\version_03\v03_shape.geojson


In [8]:
transit_net = TransitNetwork.read(feed_path = input_dir)

2021-01-26 11:47:10, INFO: Read in transit feed from: D:/gitHub/travel-model-two-networks\data\processed\version_03
2021-01-26 11:47:10, INFO: Read in transit feed from: D:/gitHub/travel-model-two-networks\data\processed\version_03
2021-01-26 11:47:12, INFO: Removing fare_attributes.txt from transit network config because file not found
2021-01-26 11:47:12, INFO: Removing fare_attributes.txt from transit network config because file not found
2021-01-26 11:47:12, INFO: Removing fare_rules.txt from transit network config because file not found
2021-01-26 11:47:12, INFO: Removing fare_rules.txt from transit network config because file not found
2021-01-26 11:47:13, INFO: Removing transfers.txt from transit network config because file not found
2021-01-26 11:47:13, INFO: Removing transfers.txt from transit network config because file not found
2021-01-26 11:47:13, WARNING: The following stop_ids from are referenced but missing from stops.txt: ['place_CVBA', 'ctsj', 'place_WOBA', 'ctgi', 'p

In [9]:
places_gdf = gpd.read_file(os.path.join(boundary_dir, 'CA_Places_TIGER2016.shp'))
places_gdf.head()

,STATEFP,PLACEFP,PLACENS,GEOID,NAME,NAMELSAD,LSAD,CLASSFP,PCICBSA,PCINECTA,MTFCC,FUNCSTAT,ALAND,AWATER,INTPTLAT,INTPTLON,geometry
0,06,66140,02411785,0666140,San Fernando,San Fernando city,25,C1,N,N,G4110,A,6148697,0,+34.2886519,-118.4362413,"POLYGON ((-13186463.855 4067122.651, -13186256..."
1,06,14190,02409487,0614190,Cloverdale,Cloverdale city,25,C1,N,N,G4110,A,7863863,59201,+38.7959624,-123.0153700,"POLYGON ((-13696207.846 4691208.338, -13696205..."
2,06,16560,02410240,0616560,Cotati,Cotati city,25,C1,N,N,G4110,A,4869007,8380,+38.3284920,-122.7100491,"POLYGON ((-13662197.838 4626644.614, -13662188..."
3,06,65042,02411779,0665042,San Buenaventura (Ventura),San Buenaventura (Ventura) city,25,C1,Y,N,G4110,A,56500370,27033715,+34.2677796,-119.2542062,"MULTIPOLYGON (((-13280094.138 4075172.878, -13..."
4,06,30014,02410601,0630014,Glendora,Glendora city,25,C1,N,N,G4110,A,50251851,403066,+34.1449667,-117.8476672,"POLYGON ((-13123577.793 4048398.248, -13123527..."


In [10]:
union_city_gdf = places_gdf.loc[places_gdf['NAME'] == 'Union City'].to_crs(4326)
union_city_gdf.head()

,STATEFP,PLACEFP,PLACENS,GEOID,NAME,NAMELSAD,LSAD,CLASSFP,PCICBSA,PCINECTA,MTFCC,FUNCSTAT,ALAND,AWATER,INTPTLAT,INTPTLON,geometry
472,06,81204,02412130,0681204,Union City,Union City city,25,C1,N,N,G4110,A,50244982,0,+37.6029595,-122.0187225,"POLYGON ((-122.09655 37.58293, -122.09648 37.5..."


### Transit Selections

In [13]:
agency_df = transit_net.feed.agency.loc[transit_net.feed.agency['agency_id'] == 'UCT']
routes_df = transit_net.feed.routes.loc[transit_net.feed.routes['agency_id'] == 'UCT']
trips_df = transit_net.feed.trips[transit_net.feed.trips['route_id'].isin(routes_df['route_id'])]
freqs_df = transit_net.feed.frequencies[transit_net.feed.frequencies['trip_id'].isin(trips_df['trip_id'])]
shapes_df = transit_net.feed.shapes[transit_net.feed.shapes['shape_id'].isin(trips_df['shape_id'])]
stop_times_df = transit_net.feed.stop_times[transit_net.feed.stop_times['trip_id'].isin(trips_df['trip_id'])]
stops_df = transit_net.feed.stops[transit_net.feed.stops['stop_id'].isin(stop_times_df['stop_id'])]

### Roadway Selection

In [11]:
shapes_gdf = road_net.shapes_df.copy()

In [17]:
shapes_gdf.info()

<class 'geopandas.geodataframe.GeoDataFrame'>
Int64Index: 1169824 entries, 0 to 1169823
Data columns (total 8 columns):
index                 1169824 non-null int64
backReferenceId       869970 non-null object
forwardReferenceId    869970 non-null object
fromIntersectionId    1147790 non-null object
id                    1169824 non-null object
shape_id              869970 non-null float64
toIntersectionId      869970 non-null object
geometry              1169824 non-null geometry
dtypes: float64(1), geometry(1), int64(1), object(5)
memory usage: 80.3+ MB


In [14]:
union_city_shapes_gdf = gpd.sjoin(shapes_gdf, union_city_gdf, op='intersects')

In [16]:
union_city_shapes_gdf.info()

<class 'geopandas.geodataframe.GeoDataFrame'>
Int64Index: 8514 entries, 47424 to 1169819
Data columns (total 25 columns):
index                 8514 non-null int64
backReferenceId       6273 non-null object
forwardReferenceId    6273 non-null object
fromIntersectionId    8274 non-null object
id                    8514 non-null object
shape_id              6273 non-null float64
toIntersectionId      6273 non-null object
geometry              8514 non-null geometry
index_right           8514 non-null int64
STATEFP               8514 non-null object
PLACEFP               8514 non-null object
PLACENS               8514 non-null object
GEOID                 8514 non-null object
NAME                  8514 non-null object
NAMELSAD              8514 non-null object
LSAD                  8514 non-null object
CLASSFP               8514 non-null object
PCICBSA               8514 non-null object
PCINECTA              8514 non-null object
MTFCC                 8514 non-null object
FUNCSTAT         

In [28]:
links_df = road_net.links_df.copy()
links_df.columns

Index(['index', 'A', 'B', 'access', 'assignable', 'bike_access', 'county',
       'distance', 'drive_access', 'egress', 'fromIntersectionId', 'ft',
       'ft_cal', 'id', 'lanes', 'locationReferences', 'managed', 'maxspeed',
       'model_link_id', 'name', 'oneWay', 'rail_only', 'rail_traveltime',
       'ref', 'roadway', 'segment_id', 'shape_id', 'shstGeometryId',
       'shstReferenceId', 'toIntersectionId', 'tollbooth', 'tollseg',
       'transit', 'u', 'useclass', 'v', 'walk_access', 'wayId', 'cntype',
       'lanes_EA', 'lanes_AM', 'lanes_MD', 'lanes_PM', 'lanes_EV',
       'ML_lanes_EA', 'ML_lanes_AM', 'ML_lanes_MD', 'ML_lanes_PM',
       'ML_lanes_EV', 'useclass_EA', 'useclass_AM', 'useclass_MD',
       'useclass_PM', 'useclass_EV', 'geometry'],
      dtype='object')

In [20]:
union_city_transit_links_df = links_df[(links_df.A.isin(shapes_df.shape_model_node_id.tolist())) | 
                                       (links_df.B.isin(shapes_df.shape_model_node_id.tolist()))].copy()

In [29]:
union_city_links_df = links_df[(links_df.model_link_id.isin(union_city_transit_links_df.model_link_id)) |
                               (links_df.shstGeometryId.isin(union_city_shapes_gdf.id))].copy()

In [38]:
union_city_links_df["bus_only"] = 0

In [30]:
union_city_links_df.shape

(16411, 55)

In [31]:
nodes_df = road_net.nodes_df.copy()
nodes_df.head()

,model_node_id,index,osm_node_id,shst_node_id,county,drive_access,walk_access,bike_access,rail_only,X,Y,farezone,geometry
model_node_id_idx,,,,,,,,,,,,,
3000000,3000000,0,2.40124e+09,505d64eb98f1da8d812a3b3801034308,Contra Costa,1,1,1,0,-122.331554,37.981204,0,POINT (-122.33155 37.98120)
3000001,3000001,1,5.78391e+07,473979c78435732f01ca5a168afb62e0,Contra Costa,1,1,1,0,-121.944770,37.953322,0,POINT (-121.94477 37.95332)
5000000,5000000,2,1.02439e+09,fc7b575d5d8c961d4a70fca846ae7f80,Marin,1,1,1,0,-122.539828,37.897999,0,POINT (-122.53983 37.89800)
2000000,2000000,3,6.55614e+07,6c60cf34e9dc3e123eefb829fe80c76a,Santa Clara,1,1,1,0,-122.031897,37.267386,1,POINT (-122.03190 37.26739)
2000001,2000001,4,4.54558e+09,013e1f994fd86c1f226098f8364f7286,Santa Clara,1,1,1,0,-122.011863,37.378447,0,POINT (-122.01186 37.37845)


In [32]:
nodes_df.info()

<class 'geopandas.geodataframe.GeoDataFrame'>
Int64Index: 696464 entries, 3000000 to 7547315
Data columns (total 13 columns):
model_node_id    696464 non-null int64
index            696464 non-null int64
osm_node_id      696464 non-null object
shst_node_id     696464 non-null object
county           696464 non-null object
drive_access     696464 non-null int64
walk_access      696464 non-null int64
bike_access      696464 non-null int64
rail_only        696464 non-null int64
X                696464 non-null float64
Y                696464 non-null float64
farezone         696464 non-null int64
geometry         696464 non-null geometry
dtypes: float64(2), geometry(1), int64(7), object(3)
memory usage: 74.4+ MB


In [33]:
union_city_nodes_df = nodes_df[nodes_df.model_node_id.isin(union_city_links_df.A.tolist() + 
                                                           union_city_links_df.B.tolist())].copy()
union_city_nodes_df.info()

<class 'geopandas.geodataframe.GeoDataFrame'>
Int64Index: 5233 entries, 2500116 to 391404
Data columns (total 13 columns):
model_node_id    5233 non-null int64
index            5233 non-null int64
osm_node_id      5233 non-null object
shst_node_id     5233 non-null object
county           5233 non-null object
drive_access     5233 non-null int64
walk_access      5233 non-null int64
bike_access      5233 non-null int64
rail_only        5233 non-null int64
X                5233 non-null float64
Y                5233 non-null float64
farezone         5233 non-null int64
geometry         5233 non-null geometry
dtypes: float64(2), geometry(1), int64(7), object(3)
memory usage: 572.4+ KB


### Make and Verify Network

In [39]:
union_city_roadway_net = copy.deepcopy(road_net)
union_city_roadway_net.links_df = union_city_links_df
union_city_roadway_net.shapes_df = union_city_shapes_gdf
union_city_roadway_net.nodes_df = union_city_nodes_df

In [35]:
union_city_transit_net =  copy.deepcopy(transit_net)
union_city_transit_net.feed.agency = agency_df
union_city_transit_net.feed.routes = routes_df
union_city_transit_net.feed.trips  = trips_df
union_city_transit_net.feed.frequencies = freqs_df
union_city_transit_net.feed.shapes = shapes_df
union_city_transit_net.feed.stop_times = stop_times_df
union_city_transit_net.feed.stops = stops_df

union_city_transit_net.road_net = union_city_roadway_net

In [40]:
union_city_roadway_net.links_df.columns

Index(['index', 'A', 'B', 'access', 'assignable', 'bike_access', 'county',
       'distance', 'drive_access', 'egress', 'fromIntersectionId', 'ft',
       'ft_cal', 'id', 'lanes', 'locationReferences', 'managed', 'maxspeed',
       'model_link_id', 'name', 'oneWay', 'rail_only', 'rail_traveltime',
       'ref', 'roadway', 'segment_id', 'shape_id', 'shstGeometryId',
       'shstReferenceId', 'toIntersectionId', 'tollbooth', 'tollseg',
       'transit', 'u', 'useclass', 'v', 'walk_access', 'wayId', 'cntype',
       'lanes_EA', 'lanes_AM', 'lanes_MD', 'lanes_PM', 'lanes_EV',
       'ML_lanes_EA', 'ML_lanes_AM', 'ML_lanes_MD', 'ML_lanes_PM',
       'ML_lanes_EV', 'useclass_EA', 'useclass_AM', 'useclass_MD',
       'useclass_PM', 'useclass_EV', 'geometry', 'bus_only'],
      dtype='object')

In [41]:
union_city_transit_net.validate_road_network_consistencies()

True

In [42]:
union_city_transit_net.validate_transit_stops()

True

# Write out

In [44]:
union_city_roadway_net.write(path = output_dir, filename = "union_city")

In [45]:
union_city_transit_net.write(path = output_dir)

2021-01-26 12:31:38, INFO: Writing transit to directory: Z:/Data/Users/Sijia/MTC/github/Lasso\examples\mtc
2021-01-26 12:31:38, INFO: Writing transit to directory: Z:/Data/Users/Sijia/MTC/github/Lasso\examples\mtc
